In [1]:
# SOLOv2 train  
from importlib import reload


from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import os


def get_timestamp():
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

cfg = Config.fromfile('./configs/solov2/solov2_light_448_r50_fpn_custom.py') #works, batch size 32
cfg.load_from = './checkpoints/epoch_4.pth' #SOLOv2_LIGHT_448_R50_3x 
# is is possible to dynamically download Pretrained model - the string should start with "modelzoo://"

# cfg = Config.fromfile('./configs/solov2/solov2_light_512_dcn_r50_fpn_custom.py')
# cfg.load_from = './checkpoints/SOLOv2_LIGHT_512_DCN_R50_3x.pth'

# cfg = Config.fromfile('./configs/solov2/solov2_light_448_r50_fpn_8gpu_3x.py') #works, batch size 12
# cfg.load_from = './checkpoints/SOLOv2_Light_448_R50_3x.pth'

cfg.dataset_type = 'CocoDataset'
PREFIX = os.path.abspath('G:/datasety/new')
cfg.data.train.ann_file = PREFIX + '/instances_hands_30600.json' # single epoch 30k for 26 mins

# PREFIX = os.path.abspath('../datasets/rgbd_joined_dataset/ruka_2')
# cfg.data.train.ann_file = PREFIX + '/instances_hands_train2022.json'

cfg.data.train.img_prefix = PREFIX + "/color/"
cfg.data.train.type = 'CocoDataset'

cfg.optimizer.lr = 0.001
cfg.lr_config.warmup = None

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device_ids = range(1)
cfg.gpus = 1

cfg.work_dir = "./checkpoints"
cfg.total_epochs = 4
cfg.checkpoint_config = dict(create_symlink=False, interval = 1)
cfg.data.imgs_per_gpu = 32
cfg.data.workers_per_gpu = 4

cfg.log_config = dict(
    interval=1,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')
    ])

model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]
datasets[0].CLASSES = ["hand", "dummy"]
model.CLASSES = datasets[0].CLASSES

# CHECK OUT frozen_stages=0 for fine-tunning !!
# try resnext backbone

train_detector(model, datasets[0], cfg, distributed=False, validate=False)
print("Training finished")

2022-03-11 15:11:58,729 - mmdet - INFO - load model from: torchvision://resnet50
2022-03-11 15:11:58,846 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 7, 7]) from checkpoint, the shape in current model is torch.Size([64, 4, 7, 7]).
unexpected key in source state_dict: fc.weight, fc.bias



loading annotations into memory...
Done (t=0.77s)
creating index...
index created!


2022-03-11 15:12:00,104 - mmdet - INFO - load checkpoint from ./checkpoints/epoch_4.pth
2022-03-11 15:12:00,468 - mmdet - INFO - Start running, host: Stefan@LAPTOP-UV8365G7, work_dir: c:\Users\Stefan\source\repos\HGR_CNN\SOLO\checkpoints
2022-03-11 15:12:00,468 - mmdet - INFO - workflow: [('train', 1)], max: 4 epochs


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "D:\APPS\conda\envs\solo\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "D:\APPS\conda\envs\solo\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\APPS\conda\envs\solo\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\Stefan\source\repos\HGR_CNN\SOLO\mmdet\datasets\custom.py", line 131, in __getitem__
    data = self.prepare_train_img(idx)
  File "c:\Users\Stefan\source\repos\HGR_CNN\SOLO\mmdet\datasets\custom.py", line 144, in prepare_train_img
    return self.pipeline(results)
  File "c:\Users\Stefan\source\repos\HGR_CNN\SOLO\mmdet\datasets\pipelines\compose.py", line 24, in __call__
    data = t(data)
  File "